In [1]:
# Import api/access_token keys from credentials.py

import re
import tweepy
import mysql.connector
import pandas as pd
from textblob import TextBlob


api_key = 'XpDoApLguOinEi9FRrpgi497P'
api_secret = 'TXqXrhYoHfMCZNVSC2oyuKO3e8Dm33AVDxEfeUsG5HYzfKM069' 
access_token = '2885239189-4D38no2Dm5YK8jsQhwzyEcOyfZs1FvB1zNK3kje'
access_token_secret ='fEvJ3YAlBNka6k5CnT9dlgSqe2fEunZNNwWUfNtKTuI6U'

auth  = tweepy.OAuthHandler(api_key, \
							api_secret)
auth.set_access_token(access_token,  \
					  access_token_secret)
api = tweepy.API(auth)

TRACK_WORDS = ['jetblue']
TABLE_NAME = "jetblue"
TABLE_ATTRIBUTES = "id_str VARCHAR(255), created_at DATETIME, text VARCHAR(255), \
            polarity INT, subjectivity INT, user_created_at VARCHAR(255), user_location VARCHAR(255), \
            user_description VARCHAR(255), user_followers_count INT, longitude DOUBLE, latitude DOUBLE, \
            retweet_count INT, favorite_count INT"


# Override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):
	'''
	Tweets are known as “status updates”. So the Status class in tweepy has properties describing the tweet.
	https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html
	'''
	
	def on_status(self, status):
		'''
		Extract info from tweets
		'''
		
		if status.retweeted:
			# Avoid retweeted info, and only original tweets will be received
			return True
		# Extract attributes from each tweet
		id_str = status.id_str
		created_at = status.created_at
		text = deEmojify(status.text)    # Pre-processing the text  
		sentiment = TextBlob(text).sentiment
		polarity = sentiment.polarity
		subjectivity = sentiment.subjectivity
		
		user_created_at = status.user.created_at
		user_location = deEmojify(status.user.location)
		user_description = deEmojify(status.user.description)
		user_followers_count =status.user.followers_count
		longitude = None
		latitude = None
		if status.coordinates:
			longitude = status.coordinates['coordinates'][0]
			latitude = status.coordinates['coordinates'][1]
			
		retweet_count = status.retweet_count
		favorite_count = status.favorite_count
		
		print(status.text)
		print("Long: {}, Lati: {}".format(longitude, latitude))
		
		# Store all data in MySQL
		if mydb.is_connected():
			mycursor = mydb.cursor()
			sql = "INSERT INTO {} (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(TABLE_NAME)
			val = (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, \
				user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count)
			mycursor.execute(sql, val)
			mydb.commit()
			mycursor.close()
	
	
	def on_error(self, status_code):
		'''
		Since Twitter API has rate limits, stop srcraping data as it exceed to the thresold.
		'''
		if status_code == 420:
			# return False to disconnect the stream
			return False

In [2]:
def clean_tweet(self, tweet): 
    ''' 
    Use sumple regex statemnents to clean tweet text by removing links and special characters
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \
                                |(\w+:\/\/\S+)", " ", tweet).split()) 
def deEmojify(text):
    '''
    Strip all non-ASCII characters to remove emoji characters
    '''
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [3]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="M@sters@21",
    database="TwitterDB",
    charset = 'utf8'
)
if mydb.is_connected():
    '''
    Check if this table exits. If not, then create a new one.
    '''
    mycursor = mydb.cursor()
    mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(TABLE_NAME))
    if mycursor.fetchone()[0] != 1:
        mycursor.execute("CREATE TABLE {} ({})".format(TABLE_NAME, TABLE_ATTRIBUTES))
        mydb.commit()
    mycursor.close()

In [4]:
auth  = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [ ]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener = myStreamListener)
myStream.filter(languages=["en"], track = TRACK_WORDS)
# Close the MySQL connection as it finished
# However, this won't be reached as the stream listener won't stop automatically
# Press STOP button to finish the process.
mydb.close()

JetBlue Airways flight #JBU101 spotted at 8,275 feet! https://t.co/ITaHKa7yTD
Long: None, Lati: None
RT @celtics: The pass... the finish... obvious choice for tonight's @jetblue Play of the Game ☘️ https://t.co/uryCKeLmtZ
Long: None, Lati: None
RT @coffeespoonie: Hey @JetBlue jsyk my wheelchair is still broken. my wheels are being scraped off into rubber dust every time I move as t…
Long: None, Lati: None
RT @choochoobear: Well this is enraging. https://t.co/34Gi7TgbZX
Long: None, Lati: None
RT @coffeespoonie: Hey @JetBlue jsyk my wheelchair is still broken. my wheels are being scraped off into rubber dust every time I move as t…
Long: None, Lati: None
@lemonmartinis I have 156,450 Jetblue miles...  

Does that count?
Long: None, Lati: None
RT @coffeespoonie: Hey @JetBlue jsyk my wheelchair is still broken. my wheels are being scraped off into rubber dust every time I move as t…
Long: None, Lati: None
RT @coffeespoonie: @JetBlue The reasoning you originally gave me for why you chose to